# sentiment analysis on Pre-Trained Network

load in a pretrained LSTM network using Tensorflow's Saver object.

creating Tensorflow graph. 

hyperperameter declaration 

In [10]:
numDimensions = 300
maxSeqLength = 250
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000

loading data structures.

In [11]:
import re
import numpy as np
wordsList = np.load('wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')

creating graph

In [12]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loading n/w

In [13]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-90000


preprocessing

In [14]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

API authentication and loading of tweets

In [15]:
import tweepy
consumer_key= 'E0pFYVai9VaOhqLiRBEC6gpGF'
consumer_secret= 'XAMh4l9XL5nwFK3MN5tAjtXA2YgDN1tw5f7L2n6dz5ib8VYlbm'

access_token='3261604734-86c7DOJP98GwNeFWzvgPQKFUTyHn1ZFwlloJP3v'
access_token_secret='eXEmlEAdxaFjueVP03jsAWeOeNMkI7ToiDQkyvLDa6eX7'


In [16]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
public_tweets = api.search('TCS', count = 1000)


In [17]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())


In [18]:
positive = 0
negative = 0

In [19]:
for tweet in public_tweets:
    a=clean_tweet(tweet.text)
    inputMatrix = getSentenceMatrix(a)
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        print ("Positive Sentiment")
        positive += 1
    else:
        print ("Negative Sentiment")
        negative += 1
    print(a)
    #text = clean_tweet(tweets.txt)
    #print(text)
    

Positive Sentiment
Yunomi feat nicamoq from SoundCloud NowPlaying
Positive Sentiment
RT First Deposit Campaign Win 50 USDT 20 Winners How to Enter 1 Follow 2 RT 3 Sign up for a new OKEx account via http
Positive Sentiment
RT RT this post and follow us for your chance to twirl into winter and win a Ballet Theatre and Ice Skating Friends We ve g
Positive Sentiment
RT RT this post and follow us for your chance to twirl into winter and win a Ballet Theatre and Ice Skating Friends We ve g
Positive Sentiment
RT RT this post and follow us for your chance to twirl into winter and win a Ballet Theatre and Ice Skating Friends We ve g
Negative Sentiment
Container Store Group TCS Trading Down 41 2 Following Weak Earnings
Positive Sentiment

Positive Sentiment
rekanan feat REFLECTION
Positive Sentiment
10 12 1548
Negative Sentiment
W12 Studios is now a part of TCS Interactive Excited
Negative Sentiment
ROSSI
Negative Sentiment
tcs
Positive Sentiment
RT RT this post and follow us for your chance to 

Positive Sentiment
RT Europe What do retailers value in their technology partners s shares her thoughts with
Negative Sentiment
RT HappyHalloween For your chance to WIN this spook tacular bundle on this wicked WinItWednesday simply FOLLOW us and RT
Negative Sentiment
RT It will focus you 3 days Right about now you start to hone down on your training you re focused on running your be
Negative Sentiment
RT na Use our 2018 TCSNYCMarathon app to create digital cheer cards and show your support for friends and family in this year s race
Negative Sentiment
RT Meet Canadian organizations innovative companies amp officials from municipalities looking at smart solutions Canada booth is
Positive Sentiment
RT ANZ After his first marathon earlier this year at the TCS Australian Running Festival in Canberra Rahul is in the US with his fel
Positive Sentiment
RT canada Attending WBFMI today Visit our booth to find out about why the time to InvestinCanada is now Milano SDC
Positive Sentiment
Best of l

In [20]:
print('Positive:', positive)
print('Negative:', negative)

Positive: 65
Negative: 35
